# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
import ROOT
import root_numpy as rnp
import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

ROOT.enableJSVis()
#ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()
    

Welcome to JupyROOT 6.14/02


In [2]:
# %load ../drawingTools


In [3]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [44]:
# %load samples.py
import ROOT
import pandas as pd
import python.selections as selections

version = 'v73t'


def get_label_dict(selections):
    dictionary = {}
    for sel in selections:
        dictionary[sel.name] = sel.label
    return dictionary


# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('all_flat5to80_PU0', 'PU0', 'v75t', 'V9'),
    Sample('all_flat5to80_PU200', 'PU200', 'v75t', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

samples_nugus = [
    Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
    ]

samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200', 'PU200', version, 'V8')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200', 'PU200', 'v75', 'V9')
    ]

all_tpsets = {'DEF': 'dRC3d',
              'DEFCalib': 'NNDR Calib v1',
              'DEFNC': 'dRC3d + new Th',
              'HMvDR': 'HistoMaxC3d + dR(layer)',
              'HMvDRNC0': 'HMC3d+dR(layer)+NC0',
              'HMvDRNC1': 'HMC3d+dR(layer)+NC1',
              'EG': 'EG',
              'TkEG': 'TkEG',
              'TkEle': 'TkEle',
              'TkIsoEle': 'TkIsoEle',
              'L1Trk': 'L1Track'}


Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU0_v73t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_photon_flat8to150_PU200_v73t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_pion_flat2to100_PU0_v73t.root does not exist
Error in <TFile::TFile>: file ../plots1/histos_nugun_alleta_pu0_v73t.root does not exist


In [45]:
# %load settings.py
# === samples =====================================================
samples = []

samples = samples_nugunrates_V8
samples += samples_nugunrates_V9


sample = 'V8'

do_rate = True

# === TP ==========================================================
tps = ['DEF', 'HMvDR', 'DEFNC', 'HMvDRNC0', 'HMvDRNC1']

# === TP selections ================================================
tp_select = {}
tp_select['DEF'] = ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD']
tp_select['DEFNC'] = tp_select['DEF']
tp_select['HMvDR'] = tp_select['DEF']
tp_select['HMvDRNC0'] = tp_select['DEF']
tp_select['HMvDRNC1'] = tp_select['DEF']


tp_select['EG'] = ['EGq2', 'EGq3']

tp_select['TkEG'] = get_label_dict(selections.tkeg_qual_selections).keys()
tp_select['TkEle'] = ['EGq2', 'EGq3']

tp_select['TkEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['TkIsoEle'] = ['EGq2', 'EGq3', 'EGq2Iso0p2', 'EGq3Iso0p2', 'EGq2Iso0p3', 'EGq3Iso0p3']
tp_select['L1Trk'] = ['all', 'Pt2', 'Pt10']

# ==== GEN selections ===============================================

gen_select = ['GENEtaBC']
# gen_select = ['GEN', 'GENEtaA', 'GENEtaB', 'GENEtaC', 'GENEtaD', 'GENEtaE',
#               'GENEtaAB', 'GENEtaABC', 'GENEtaBC', 'GENEtaBCD', 'GENEtaBCDE' ]

# tp_select['TkEG'] = ['EGq2EtaBC', 'EGq3EtaBC', 'EGq2EtaBCM2', 'EGq3EtaBCM2', 'EGq2EtaBCM2s', 'EGq3EtaBCM2s','EGq2EtaBCM3', 'EGq3EtaBCM3', 'EGq2EtaBCM3s', 'EGq3EtaBCM3s','EGq2EtaBCM4', 'EGq3EtaBCM4']


# ==== adapt the plot retrieval
tpsets = {}
tpset_selections = {}
gen_selections = {}

for tp in tps:
    tpsets[tp] = all_tpsets[tp]

if 'DEF' in tps or 'HMvDR' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tp_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tp_match_selections))
if 'EG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.eg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.eg_pt_selections))
if 'TkEG' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkeg_pt_selections))
if 'TkEle' in tps or 'TkIsoEle' in tps:
    if do_rate:
        tpset_selections.update(get_label_dict(selections.tkisoeg_rate_selections))
    else:
        tpset_selections.update(get_label_dict(selections.tkisoeg_pt_selections))
if 'L1Trk' in tps:
    tpset_selections.update(get_label_dict(selections.tracks_selections))


gen_selections.update(get_label_dict(selections.gen_part_selections))
gen_selections.update({'nomatch': ''})

import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
print gen_select


--- TPs: 
['DEF', 'HMvDR', 'DEFNC', 'HMvDRNC0', 'HMvDRNC1']
--- TP selections:
{   'DEF': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'DEFNC': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'EG': ['EGq2', 'EGq3'],
    'HMvDR': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'HMvDRNC0': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'HMvDRNC1': ['all', 'EtaBC', 'Em', 'EmEtaBC', 'EmEtaD'],
    'L1Trk': ['all', 'Pt2', 'Pt10'],
    'TkEG': [   'EGq2M1P2',
                'EGq2M1P2S',
                'EGq3M1P10',
                'EGq2M1P5',
                'EGq2M1P10',
                'EGq3M1',
                'EGq3M1P5S',
                'EGq3',
                'EGq2',
                'EGq3M1P2S',
                'EGq2M1P5S',
                'EGq2M1P10S',
                'EGq3M1P2',
                'EGq2M1',
                'EGq3M1P5',
                'EGq3M1P10S'],
    'TkEle': [   'EGq2',
                 'EGq3',
                 'EGq2Iso0p2',
                 'EGq3Iso0p2',
   

In [13]:
tpset_selections


{'Em': 'EGId',
 'EmEtaB': 'EGId, 1.52 < |#eta^{L1}| <= 1.7',
 'EmEtaBC': 'EGId, 1.52 < |#eta^{L1}| <= 2.4',
 'EmEtaBCD': 'EGId, 1.52 < |#eta^{L1}| <= 2.8',
 'EmEtaC': 'EGId, 1.7 < |#eta^{L1}| <= 2.4',
 'EmEtaD': 'EGId, 2.4 < |#eta^{L1}| <= 2.8',
 'EtaB': '1.52 < |#eta^{L1}| <= 1.7',
 'EtaBC': '1.52 < |#eta^{L1}| <= 2.4',
 'EtaBCD': '1.52 < |#eta^{L1}| <= 2.8',
 'EtaC': '1.7 < |#eta^{L1}| <= 2.4',
 'EtaD': '2.4 < |#eta^{L1}| <= 2.8',
 'all': ''}

In [14]:
gen_selections

{'GEN': '',
 'GENEtaB': '1.52 < |#eta^{GEN}| <= 1.7',
 'GENEtaBC': '1.52 < |#eta^{GEN}| <= 2.4',
 'GENEtaBCD': '1.52 < |#eta^{GEN}| <= 2.8',
 'GENEtaC': '1.7 < |#eta^{GEN}| <= 2.4',
 'GENEtaD': '2.4 < |#eta^{GEN}| <= 2.8',
 'GENPt10': 'p_{T}^{GEN}>=10GeV',
 'GENPt20': 'p_{T}^{GEN}>=20GeV',
 'GENPt30': 'p_{T}^{GEN}>=30GeV',
 'GENPt40': 'p_{T}^{GEN}>=40GeV',
 'nomatch': ''}

In [17]:
tpset_selections

{'Em': 'EGId',
 'EmEtaB': 'EGId, 1.52 < |#eta^{L1}| <= 1.7',
 'EmEtaBC': 'EGId, 1.52 < |#eta^{L1}| <= 2.4',
 'EmEtaBCD': 'EGId, 1.52 < |#eta^{L1}| <= 2.8',
 'EmEtaC': 'EGId, 1.7 < |#eta^{L1}| <= 2.4',
 'EmEtaD': 'EGId, 2.4 < |#eta^{L1}| <= 2.8',
 'EtaB': '1.52 < |#eta^{L1}| <= 1.7',
 'EtaBC': '1.52 < |#eta^{L1}| <= 2.4',
 'EtaBCD': '1.52 < |#eta^{L1}| <= 2.8',
 'EtaC': '1.7 < |#eta^{L1}| <= 2.4',
 'EtaD': '2.4 < |#eta^{L1}| <= 2.8',
 'all': ''}

In [46]:
%%time

hplot = HPlot(samples, tpsets, tpset_selections, [])


hplot.cache_histo(classtype=histos.RateHistos, 
                  samples=samples,
                  pus=[],
                  tps=tpsets,
                  tp_sels=tpset_selections,
                  gen_sels=None)



<__main__.Sample instance at 0x110115b90>
<__main__.Sample instance at 0x110115b90> HMvDR Em None
<__main__.Sample instance at 0x110115b90> HMvDR EtaD None
<__main__.Sample instance at 0x110115b90> HMvDR all None
<__main__.Sample instance at 0x110115b90> HMvDR EmEtaBC None
<__main__.Sample instance at 0x110115b90> HMvDR EmEtaBCD None
<__main__.Sample instance at 0x110115b90> HMvDR EmEtaB None
<__main__.Sample instance at 0x110115b90> HMvDR EmEtaC None
<__main__.Sample instance at 0x110115b90> HMvDR EtaBC None
<__main__.Sample instance at 0x110115b90> HMvDR EmEtaD None
<__main__.Sample instance at 0x110115b90> HMvDR EtaB None
<__main__.Sample instance at 0x110115b90> HMvDR EtaC None
<__main__.Sample instance at 0x110115b90> HMvDR EtaBCD None
<__main__.Sample instance at 0x110115b90> HMvDRNC0 Em None
<__main__.Sample instance at 0x110115b90> HMvDRNC0 EtaD None
<__main__.Sample instance at 0x110115b90> HMvDRNC0 all None
<__main__.Sample instance at 0x110115b90> HMvDRNC0 EmEtaBC None
<__ma

In [24]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,V8,PU200,HMvDR,Em,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x1100d5cf8>
1,V8,PU200,HMvDR,EtaD,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x110134b00>
2,V8,PU200,HMvDR,all,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x1100d5c20>
3,V8,PU200,HMvDR,EmEtaBC,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x1100495a8>
4,V8,PU200,HMvDR,EmEtaBCD,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x110134560>
5,V8,PU200,HMvDR,EmEtaB,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x1100d5cb0>
6,V8,PU200,HMvDR,EmEtaC,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x110134b90>
7,V8,PU200,HMvDR,EtaBC,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x1101349e0>
8,V8,PU200,HMvDR,EmEtaD,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x1065b5200>
9,V8,PU200,HMvDR,EtaB,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x1101347e8>


In [26]:
for tp in tps:
    for tp_sel in tpset_selections:
        print tp,tp_sel
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)


DEF Em
normalize to 31000
DEF EtaD
normalize to 31000
DEF all
normalize to 31000
DEF EmEtaBC
normalize to 31000
DEF EmEtaBCD
normalize to 31000
DEF EmEtaB
normalize to 31000
DEF EmEtaC
normalize to 31000
DEF EtaBC
normalize to 31000
DEF EmEtaD
normalize to 31000
DEF EtaB
normalize to 31000
DEF EtaC
normalize to 31000
DEF EtaBCD
normalize to 31000
HMvDR Em
normalize to 31000
HMvDR EtaD
normalize to 31000
HMvDR all
normalize to 31000
HMvDR EmEtaBC
normalize to 31000
HMvDR EmEtaBCD
normalize to 31000
HMvDR EmEtaB
normalize to 31000
HMvDR EmEtaC
normalize to 31000
HMvDR EtaBC
normalize to 31000
HMvDR EmEtaD
normalize to 31000
HMvDR EtaB
normalize to 31000
HMvDR EtaC
normalize to 31000
HMvDR EtaBCD
normalize to 31000
DEFNC Em


AttributeError: RateHistos instance has no attribute 'h_norm'

#### 3D clusters matched to GEN

In [29]:
for tp in ['DEF' , 'HMvDR']:
    for tp_sel in tp_select[tp]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_norm for his in hsets], labels, text=text)
         

In [32]:
for tp in ['DEF' , 'HMvDR']:
    for tp_sel in tp_select[tp]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_pt for his in hsets], labels, text=text, logy=True)
         

In [48]:
for tp_sel in tp_select['DEF']:
    hsets, labels, text = hplot.get_histo(histos.RateHistos, ['V8', 'V9'], 'PU200', ['DEF','HMvDR'], tp_sel, None)
    for hist in hsets:
        hist.normalize(31000)
    draw([his.h_pt for his in hsets], labels, text=text, logy=True)
     

normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000
normalize to 31000


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [38]:
for tp in tps:
    hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', tp, ['EmEtaABC', 'EmEtaBC'], None)
    # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
    hist_list = [his.h_pt for his in hsets]
    draw(hist_list, labels, text=text, logy=True, do_ratio=True)
#     hRatio = hist_list[1].Clone(uuid.uuid4().hex[:6])
#     stuff.append(hRatio)
#     hRatio.Divide(hist_list[0])
#     drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

c_732713 c_732713 800 800
0 
[<ROOT.TH1F object ("DEF_EmEtaBC_pt") at 0x7fb57d845d10>]


IndexError: list index out of range

In [15]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', ['DEF','DEFCalib'], 'EmEtaBC', None)
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
hist_list = [his.h_pt for his in hsets]
drawSame(hist_list, labels, text=text, logy=True)
hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
stuff.append(hRatio)
hRatio.Divide(hist_list[1])
drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [39]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', ['DEF','DEFMerged'], 'EmEtaBC', None)
# drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
hist_list = [his.h_pt for his in hsets]
draw(hist_list, labels, text=text, logy=True, do_ratio = True)
# hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
# stuff.append(hRatio)
# hRatio.Divide(hist_list[1])
# drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

c_67fe02 c_67fe02 800 800
0 
1 same,
[<ROOT.TH1F object ("DEFMerged_EmEtaBC_pt") at 0x7fb57e908670>, <ROOT.TH1F object ("DEF_EmEtaBC_pt") at 0x7fb57d845d10>]
0.623742461205 36563.409375


<ROOT.TCanvas object ("c_67fe02") at 0x7fb57e915330>

### Comparisono baseline vs Manos EGID

In [18]:
for tp in ['DEF']:
    for etaregion in [['EmEtaBC', 'Emv1EtaBC']]:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample_type, 'PU200', tp, etaregion, None)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        hist_list = [his.h_pt for his in hsets]
        drawSame(hist_list, labels, text=text, logy=True)
        hRatio = hist_list[0].Clone(uuid.uuid4().hex[:6])
        stuff.append(hRatio)
        hRatio.Divide(hist_list[1])
        drawSame([hRatio], [''], min_y=0, max_y=2, y_axis_label='ratio' )

In [19]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        # drawSame([his.h_ptVabseta for his in hsets], labels, text=text)
  

In [20]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 11) for his in hsets], labels, text=text+', p_{T}^{L1}>10GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')



In [21]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 21) for his in hsets], labels, text=text+', p_{T}^{L1}>20GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [22]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 31) for his in hsets], labels, text=text+', p_{T}^{L1}>30GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')


In [23]:
for tp in tps:
    for tp_sel in ['all', 'Em', 'Emv1']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, None, 'PU200', tp, tp_sel, None)
        # drawSeveral([his.h_ptVabseta for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        drawSame([his.h_ptVabseta.ProjectionX(uuid.uuid4().hex[:6], 41) for his in hsets], labels, text=text+', p_{T}^{L1}>40GeV', min_y=10, max_y=500, logy=True, y_axis_label='d rate/d#eta [kHz/0.05]')
